In [1]:
import pandas as pd 
import numpy as np
import os
import glob
from tqdm import tqdm
import matplotlib.pyplot as plt
import altair as alt


C:\Users\Zach\Anaconda3\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


## Function to get the boxscores with the starters

In [3]:

player_path = '..\\..\\src\\sample_player_boxscores\\' 

def boxscore_and_starters (player_path, year): 
    season_boxscore_df = pd.read_csv('{}{}_selected_player_boxscores.csv'.format(player_path,year),index_col = 0)
    ### split columns with a dash in between the statistic
    season_boxscore_df['FGM'], season_boxscore_df['FGA'] = zip(*season_boxscore_df['fg'].str.split('-').tolist())

    season_boxscore_df = season_boxscore_df.loc[:,~ season_boxscore_df.columns.isin(["fg"])] 

    season_boxscore_df['3PM'], season_boxscore_df['3PA'] =zip(*season_boxscore_df['fg3'].str.split('-').tolist())
    season_boxscore_df = season_boxscore_df.loc[:,~ season_boxscore_df.columns.isin(["fg3"])]

    season_boxscore_df['FTM'], season_boxscore_df['FTA'] =zip(*season_boxscore_df['ft'].str.split('-').tolist())
    season_boxscore_df = season_boxscore_df.loc[:,~ season_boxscore_df.columns.isin(["ft"])]


    ### Empty rows in 2022
    season_boxscore_df = season_boxscore_df[season_boxscore_df["min"]!='--']

    for var in ['FGM','FGA','3PM',"3PA" ,'FTM','FTA']:
        season_boxscore_df[var]=season_boxscore_df[var].astype(int)  
        season_boxscore_df.loc[:,"game_date"] = pd.to_datetime(season_boxscore_df.loc[:,"game_date"])

        #### Grabs the starting lineupes for each game 
        starters_season_boxscore_df =season_boxscore_df[season_boxscore_df.starter==True]
        starters_each_game_df = starters_season_boxscore_df.loc[:,["team_short_display_name","game_id","athlete_display_name"]].groupby(["team_short_display_name","game_id"]).agg(['unique']).reset_index()
        starters_each_game_df.columns = starters_each_game_df.columns.droplevel(1)
        starters_each_game_df= starters_each_game_df.rename(columns= {"athlete_display_name":"starting_lineup"})
        starters_each_game_df['starting_lineup']=starters_each_game_df.starting_lineup.apply(set)
    merged_lineups_and_boxscore_df = pd.merge(starters_each_game_df,season_boxscore_df, on = ["game_id","team_short_display_name"])
    return merged_lineups_and_boxscore_df

merged_lineups_and_boxscore_df = boxscore_and_starters(player_path,2019)
merged_lineups_and_boxscore_df


C:\Users\Zach\Anaconda3\lib\site-packages\ipykernel_launcher.py:40: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.


,team_short_display_name,game_id,starting_lineup,athlete_display_name,min,oreb,dreb,reb,ast,stl,...,team_color,season,season_type,game_date,FGM,FGA,3PM,3PA,FTM,FTA
0,Abil Christian,401083482,"{Jaylen Franklin, Jaren Lewis, Payten Ricks, T...",Jaren Lewis,31,2,2,4,4,1,...,4e2683,2019,2,2018-12-16,4,11,0,0,1,1
1,Abil Christian,401083482,"{Jaylen Franklin, Jaren Lewis, Payten Ricks, T...",Jalone Friday,20,1,6,7,1,1,...,4e2683,2019,2,2018-12-16,4,10,2,4,2,5
2,Abil Christian,401083482,"{Jaylen Franklin, Jaren Lewis, Payten Ricks, T...",Trey Lenox,24,0,1,1,0,0,...,4e2683,2019,2,2018-12-16,3,4,2,2,0,0
3,Abil Christian,401083482,"{Jaylen Franklin, Jaren Lewis, Payten Ricks, T...",Payten Ricks,31,0,2,2,1,2,...,4e2683,2019,2,2018-12-16,0,7,0,5,4,4
4,Abil Christian,401083482,"{Jaylen Franklin, Jaren Lewis, Payten Ricks, T...",Jaylen Franklin,35,0,0,0,2,0,...,4e2683,2019,2,2018-12-16,3,7,1,1,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25867,Xavier,401123723,"{Paul Scruggs, Zach Hankins, Quentin Goodin, N...",Zak Swetye,1,0,0,0,0,0,...,002144,2019,3,2019-03-20,0,0,0,0,0,0
25868,Xavier,401123723,"{Paul Scruggs, Zach Hankins, Quentin Goodin, N...",Ramon Singh,1,1,0,1,0,0,...,002144,2019,3,2019-03-20,0,0,0,0,0,0
25869,Xavier,401123723,"{Paul Scruggs, Zach Hankins, Quentin Goodin, N...",Elias Harden,12,0,2,2,0,0,...,002144,2019,3,2019-03-20,3,6,2,4,0,0
25870,Xavier,401123723,"{Paul Scruggs, Zach Hankins, Quentin Goodin, N...",Leighton Schrand,1,0,0,0,0,0,...,002144,2019,3,2019-03-20,0,0,0,0,0,0


Merge with Team boxscore to get results of each game - Missing Results from team boxscores folder - used largest_lead instead

In [4]:
def merge_team_box_score (merged_lineups_and_boxscore_df,path,year):
    team_boxscore_df = pd.read_csv('{}{}_selected_team_boxscores.csv'.format(path,year))
    ### Potentially use largest lead as outlier
    if "largest_lead" in team_boxscore_df.columns:
        columns = ["largest_lead","home_away","opponent_id","opponent_abbrev","game_id","team_abbreviation"]
    else:
        columns = ["home_away","opponent_id","opponent_abbrev","game_id","team_abbreviation"]
    simplified_team_box_score = team_boxscore_df.loc[:,columns]
    merged_df = pd.merge(merged_lineups_and_boxscore_df,simplified_team_box_score,on = ["game_id","team_abbreviation"])
    return merged_df 


In [9]:
year = 2019
teams_path = '..\\src\\sample_team_boxscores\\'

players_df = boxscore_and_starters(player_path,year)
merged_df = merge_team_box_score(players_df,teams_path,year)
merged_df

C:\Users\Zach\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  after removing the cwd from sys.path.


,team_short_display_name,game_id,starting_lineup,athlete_display_name,min,oreb,dreb,reb,ast,stl,...,FGM,FGA,3PM,3PA,FTM,FTA,largest_lead,home_away,opponent_id,opponent_abbrev
0,Abil Christian,401083482,"{Jaylen Franklin, Jaren Lewis, Payten Ricks, T...",Jaren Lewis,31,2,2,4,4,1,...,4,11,0,0,1,1,3.0,HOME,2641,TTU
1,Abil Christian,401083482,"{Jaylen Franklin, Jaren Lewis, Payten Ricks, T...",Jalone Friday,20,1,6,7,1,1,...,4,10,2,4,2,5,3.0,HOME,2641,TTU
2,Abil Christian,401083482,"{Jaylen Franklin, Jaren Lewis, Payten Ricks, T...",Trey Lenox,24,0,1,1,0,0,...,3,4,2,2,0,0,3.0,HOME,2641,TTU
3,Abil Christian,401083482,"{Jaylen Franklin, Jaren Lewis, Payten Ricks, T...",Payten Ricks,31,0,2,2,1,2,...,0,7,0,5,4,4,3.0,HOME,2641,TTU
4,Abil Christian,401083482,"{Jaylen Franklin, Jaren Lewis, Payten Ricks, T...",Jaylen Franklin,35,0,0,0,2,0,...,3,7,1,1,1,3,3.0,HOME,2641,TTU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25867,Xavier,401123723,"{Paul Scruggs, Zach Hankins, Quentin Goodin, N...",Zak Swetye,1,0,0,0,0,0,...,0,0,0,0,0,0,20.0,AWAY,2649,TOL
25868,Xavier,401123723,"{Paul Scruggs, Zach Hankins, Quentin Goodin, N...",Ramon Singh,1,1,0,1,0,0,...,0,0,0,0,0,0,20.0,AWAY,2649,TOL
25869,Xavier,401123723,"{Paul Scruggs, Zach Hankins, Quentin Goodin, N...",Elias Harden,12,0,2,2,0,0,...,3,6,2,4,0,0,20.0,AWAY,2649,TOL
25870,Xavier,401123723,"{Paul Scruggs, Zach Hankins, Quentin Goodin, N...",Leighton Schrand,1,0,0,0,0,0,...,0,0,0,0,0,0,20.0,AWAY,2649,TOL


In [8]:
#### For saving 


# year = 2019
# for year in np.arange(2019,2023):
#     print(year)
#     players_df = boxscore_and_starters(player_path,year)
#     merged_df = merge_team_box_score(players_df,teams_path,year)
#     merged_df.to_csv("src\\team_player_boxscore_merged\\{}.csv".format(year))
# #     display(merged_df.head())


Starting Lineup Merged With BoxScores Averages - Further Analysis

In [30]:

# #### Merge the starters_each_game_df with season_boxscore_df (all players) on game_id how = (right - season_boxscores_df), then groupby team, lineup (as a set) and player to get the averages. 

# def get_averages_with_each_lineup(merged_lineups_and_boxscore_df):
#   #### Allows the startinglineup column to be included in the dataframe - makes the set hashable
#   merged_lineups_and_boxscore_df['starting_lineup']= [frozenset(s) for s in merged_lineups_and_boxscore_df['starting_lineup']]

#   #### Adjust columns starting with "pts" to include differnet columsn from the boxscore dataframe
#   averages_with_unique_LU = merged_lineups_and_boxscore_df.groupby(["team_short_display_name","starting_lineup","athlete_display_name"])[["pts",'reb', 'ast', 'stl', 'blk', 'to', 'pf',"min","season"]].mean().round(2).reset_index()

#   ### Calculate the number of games played by each player by counting the number of gamelogs (using the column points because if they played it will show as a 0 not a nan - some issues with min column)
#   ### Includes players who did not get any playing time/minutes 

#   games_with_line_up_df = merged_lineups_and_boxscore_df.groupby(["team_short_display_name","starting_lineup","athlete_display_name"])["pts"].count().reset_index().rename(columns = {"pts":"Player Games with Lineup"})
#   averages_with_unique_LU = pd.merge(averages_with_unique_LU,games_with_line_up_df,on = ["team_short_display_name","starting_lineup","athlete_display_name"])
#   averages_with_unique_LU["season"]=averages_with_unique_LU["season"].apply(int)

#   return averages_with_unique_LU
# avgs_with_lineup_df = get_averages_with_each_lineup(boxscore_and_starters(player_path,2021)).sort_values(by = 'Player Games with Lineup', ascending = False)
# avgs_with_lineup_df

In [14]:
# for yr in range(2005,2023):
#   print('Different unique number of games with a lineup in {}: {}'.format(yr, sorted(list(get_averages_with_each_lineup(boxscore_and_starters(player_path,yr))['Player Games with Lineup'].unique()))))

In [15]:
# avgs_with_lineup_df = get_averages_with_each_lineup(boxscore_and_starters(player_path,2015))
# avgs_with_lineup_df[avgs_with_lineup_df['Player Games with Lineup']==35]